#Import Dependencies


In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout, BatchNormalization
from keras.regularizers import l2
from keras.optimizers import SGD, Adam
from keras.applications import VGG16
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, balanced_accuracy_score
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#Load Dataset

In [2]:
train_data=pd.read_csv('/content/drive/MyDrive/AI  and  Machine Learning /Assignment/Assignment1/sign_mnist_train.csv')
test_data=pd.read_csv('/content/drive/MyDrive/AI  and  Machine Learning /Assignment/Assignment1/sign_mnist_test.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/AI  and  Machine Learning /Assignment/Assignment1/sign_mnist_train.csv'

#Check for class imbalance

In [ ]:

print("Training data class distribution:")
train_class_counts = train_data['label'].value_counts().sort_index()
print(train_class_counts)

print("\nTest data class distribution:")
test_class_counts = test_data['label'].value_counts().sort_index()
print(test_class_counts)

# Visualize class distribution

In [ ]:

plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
train_class_counts.plot(kind='bar')
plt.title('Training Data Class Distribution')
plt.xlabel('Class Label')
plt.ylabel('Number of Samples')

plt.subplot(1, 2, 2)
test_class_counts.plot(kind='bar')
plt.title('Test Data Class Distribution')
plt.xlabel('Class Label')
plt.ylabel('Number of Samples')
plt.tight_layout()
plt.show()

#Separate labels and features

In [ ]:

train_labels_original = train_data['label'].copy()
train_labels = train_data['label']
train_data = train_data.drop('label', axis=1)
test_labels = test_data['label']
test_data = test_data.drop('label', axis=1)


# Normalize pixel values to be between 0 and 1

In [ ]:

train_data = train_data / 255.0
test_data = test_data / 255.0


# Reshape data for CNN input (assuming images are 28x28)

In [ ]:

train_data = train_data.values.reshape(-1, 28, 28, 1)
test_data = test_data.values.reshape(-1, 28, 28, 1)

# One-hot encode the labels

In [ ]:

label_binarizer = LabelBinarizer()
train_labels = label_binarizer.fit_transform(train_labels)
test_labels = label_binarizer.transform(test_labels)

# Split training data into training and validation sets

In [ ]:

X_train, X_val, y_train, y_val = train_test_split(
    train_data, train_labels, test_size=0.2, random_state=42
)


# Calculate class weights to handle imbalance

In [ ]:

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels_original),
    y=train_labels_original
)

# Convert to dictionary for Keras

In [ ]:

class_weight_dict = {i: weight for i, weight in enumerate(class_weights)}
print("Class weights:", class_weight_dict)

# Data augmentation for training

In [ ]:

datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1
)
datagen.fit(X_train)

# Function to plot training history

In [ ]:

def plot_history(history, title='Model Performance'):
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'{title} - Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'{title} - Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Function to evaluate and visualize model performance

In [ ]:

def evaluate_model(model, X_test, y_test, model_name="Model"):
    # Evaluate the model
    test_loss, test_acc = model.evaluate(X_test, y_test)
    print(f"{model_name} - Test accuracy: {test_acc:.4f}")

    # Make predictions
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_test, axis=1)

    # Calculate balanced accuracy
    balanced_acc = balanced_accuracy_score(y_true_classes, y_pred_classes)
    print(f"{model_name} - Balanced accuracy: {balanced_acc:.4f}")

    # Classification report
    print(f"\n{model_name} - Classification Report:")
    print(classification_report(y_true_classes, y_pred_classes, digits=4))

    # Confusion matrix
    plt.figure(figsize=(12, 10))
    conf_matrix = tf.math.confusion_matrix(y_true_classes, y_pred_classes)
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title(f'{model_name} - Confusion Matrix')
    plt.show()

    return test_acc, balanced_acc, y_pred_classes

# Visualize some predictions

In [ ]:

def plot_sample_predictions(X, y_true, y_pred, n=5, model_name="Model"):
    plt.figure(figsize=(15, 3))
    indices = np.random.choice(range(len(X)), n, replace=False)
    for i, idx in enumerate(indices):
        plt.subplot(1, n, i+1)
        plt.imshow(X[idx].reshape(28, 28), cmap='gray')
        plt.title(f"True: {y_true[idx]}\nPred: {y_pred[idx]}")
        plt.axis('off')
    plt.suptitle(f"{model_name} - Sample Predictions")
    plt.tight_layout()
    plt.show()

# Get class names (0-9 and A-Z without J and Z which aren't in the dataset)

In [ ]:

class_names = list(range(10)) + [chr(i) for i in range(ord('A'), ord('Z')+1) if chr(i) not in ['J', 'Z']]
# Number of classes
num_classes = len(np.unique(np.argmax(train_labels, axis=1)))
print(f"Number of classes: {num_classes}")


# ================ BASELINE CNN MODEL ================

In [ ]:

print("\n========== BASELINE CNN MODEL WITH CLASS WEIGHTS ==========")

# Build the baseline CNN model
baseline_model = Sequential()

# First Convolutional Block
baseline_model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(28, 28, 1)))
baseline_model.add(MaxPool2D(pool_size=(2, 2)))
baseline_model.add(Dropout(0.25))

# Second Convolutional Block
baseline_model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
baseline_model.add(MaxPool2D(pool_size=(2, 2)))
baseline_model.add(Dropout(0.25))

# Third Convolutional Block
baseline_model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
baseline_model.add(MaxPool2D(pool_size=(2, 2)))
baseline_model.add(Dropout(0.4))

# Fully Connected Layers
baseline_model.add(Flatten())
baseline_model.add(Dense(512, activation='relu'))
baseline_model.add(Dropout(0.4))
baseline_model.add(Dense(num_classes, activation='softmax'))

# Compile the model
baseline_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Display model summary
print("Baseline CNN Model Summary:")
baseline_model.summary()

# Early stopping to prevent overfitting

In [ ]:

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)


# Learning rate reduction

In [ ]:

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=5,
    min_lr=0.00001
)

# Train the baseline model with class weights

In [ ]:

baseline_history = baseline_model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    epochs=50,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, reduce_lr],
    class_weight=class_weight_dict  # Add class weights to handle imbalance
)

In [ ]:
# Plot training history
plot_history(baseline_history, title="Baseline CNN")

# Evaluate baseline model

In [ ]:

baseline_acc, baseline_balanced_acc, baseline_pred_classes = evaluate_model(
    baseline_model, test_data, test_labels, "Baseline CNN with Class Weights"
)

# Visualize some predictions

In [ ]:

plot_sample_predictions(
    test_data,
    [class_names[i] for i in np.argmax(test_labels, axis=1)],
    [class_names[i] for i in baseline_pred_classes],
    model_name="Baseline CNN with Class Weights"
)

# Save the baseline model

In [ ]:

baseline_model.save('c:/Users/Acer/Downloads/sign_mnist_test.csv/baseline_cnn_model.h5')
print("Baseline model saved successfully!")

# ======= DEEPER CNN MODEL WITH REGULARIZATION =======

In [ ]:

print("\n========== DEEPER CNN MODEL WITH REGULARIZATION ==========")

# Build a deeper CNN model with regularization
deeper_model = Sequential()

# First Convolutional Block with BatchNorm and L2 regularization
deeper_model.add(Conv2D(32, (3, 3), padding='same', activation='relu',
                        kernel_regularizer=l2(0.001), input_shape=(28, 28, 1)))
deeper_model.add(BatchNormalization())
deeper_model.add(Conv2D(32, (3, 3), padding='same', activation='relu',
                        kernel_regularizer=l2(0.001)))
deeper_model.add(BatchNormalization())
deeper_model.add(MaxPool2D(pool_size=(2, 2)))
deeper_model.add(Dropout(0.25))

# Second Convolutional Block
deeper_model.add(Conv2D(64, (3, 3), padding='same', activation='relu',
                        kernel_regularizer=l2(0.001)))
deeper_model.add(BatchNormalization())
deeper_model.add(Conv2D(64, (3, 3), padding='same', activation='relu',
                        kernel_regularizer=l2(0.001)))
deeper_model.add(BatchNormalization())
deeper_model.add(MaxPool2D(pool_size=(2, 2)))
deeper_model.add(Dropout(0.25))

# Third Convolutional Block
deeper_model.add(Conv2D(128, (3, 3), padding='same', activation='relu',
                        kernel_regularizer=l2(0.001)))
deeper_model.add(BatchNormalization())
deeper_model.add(Conv2D(128, (3, 3), padding='same', activation='relu',
                        kernel_regularizer=l2(0.001)))
deeper_model.add(BatchNormalization())
deeper_model.add(MaxPool2D(pool_size=(2, 2)))
deeper_model.add(Dropout(0.4))

# Fully Connected Layers
deeper_model.add(Flatten())
deeper_model.add(Dense(512, activation='relu', kernel_regularizer=l2(0.001)))
deeper_model.add(BatchNormalization())
deeper_model.add(Dropout(0.5))
deeper_model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.001)))
deeper_model.add(BatchNormalization())
deeper_model.add(Dropout(0.5))
deeper_model.add(Dense(num_classes, activation='softmax'))

# Compile the model
deeper_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Display model summary
print("Deeper CNN Model Summary:")
deeper_model.summary()

In [ ]:
# Model checkpoint
checkpoint = ModelCheckpoint(
    'c:/Users/Acer/Downloads/sign_mnist_test.csv/deeper_model_checkpoint.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max'
)

# Train the deeper model with class weights

In [ ]:

deeper_history = deeper_model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    epochs=50,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, reduce_lr, checkpoint],
    class_weight=class_weight_dict  # Add class weights to handle imbalance
)

# Plot training history

In [ ]:

plot_history(deeper_history, title="Deeper CNN with Regularization")

# Evaluate deeper model

In [ ]:

deeper_acc, deeper_balanced_acc, deeper_pred_classes = evaluate_model(
    deeper_model, test_data, test_labels, "Deeper CNN with Regularization"
)

# Visualize some predictions

In [ ]:

plot_sample_predictions(
    test_data,
    [class_names[i] for i in np.argmax(test_labels, axis=1)],
    [class_names[i] for i in deeper_pred_classes],
    model_name="Deeper CNN with Regularization"
)

# Save the deeper model

In [ ]:

deeper_model.save('c:/Users/Acer/Downloads/sign_mnist_test.csv/deeper_cnn_model.h5')
print("Deeper model saved successfully!")

# ================ OPTIMIZER COMPARISON (SGD VS ADAM) ================

In [ ]:

print("\n========== OPTIMIZER COMPARISON (SGD VS ADAM) ==========")

# Create a model for SGD optimizer
sgd_model = Sequential()

# First Convolutional Block
sgd_model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(28, 28, 1)))
sgd_model.add(BatchNormalization())
sgd_model.add(MaxPool2D(pool_size=(2, 2)))
sgd_model.add(Dropout(0.25))

# Second Convolutional Block
sgd_model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
sgd_model.add(BatchNormalization())
sgd_model.add(MaxPool2D(pool_size=(2, 2)))
sgd_model.add(Dropout(0.25))

# Third Convolutional Block
sgd_model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
sgd_model.add(BatchNormalization())
sgd_model.add(MaxPool2D(pool_size=(2, 2)))
sgd_model.add(Dropout(0.4))

# Fully Connected Layers
sgd_model.add(Flatten())
sgd_model.add(Dense(512, activation='relu'))
sgd_model.add(BatchNormalization())
sgd_model.add(Dropout(0.4))
sgd_model.add(Dense(num_classes, activation='softmax'))

# Compile with SGD optimizer
sgd_optimizer = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
sgd_model.compile(
    optimizer=sgd_optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Display model summary
print("SGD Model Summary:")
sgd_model.summary()

# Train with SGD optimizer and class weights

In [ ]:

sgd_history = sgd_model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    epochs=50,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, reduce_lr],
    class_weight=class_weight_dict  # Add class weights to handle imbalance
)


# Plot training history


In [ ]:

plot_history(sgd_history, title="CNN with SGD Optimizer")


# Evaluate SGD model

In [ ]:

sgd_acc, sgd_balanced_acc, sgd_pred_classes = evaluate_model(
    sgd_model, test_data, test_labels, "CNN with SGD Optimizer"
)

# Compare optimizers

In [ ]:

print("\nOptimizer Comparison:")
print(f"SGD Optimizer Accuracy: {sgd_acc:.4f}, Balanced Accuracy: {sgd_balanced_acc:.4f}")
print(f"Adam Optimizer Accuracy: {baseline_acc:.4f}, Balanced Accuracy: {baseline_balanced_acc:.4f}")

# Plot optimizer comparison

In [ ]:

plt.figure(figsize=(10, 6))
x = ['SGD', 'Adam']
acc_values = [sgd_acc, baseline_acc]
balanced_acc_values = [sgd_balanced_acc, baseline_balanced_acc]

x_pos = np.arange(len(x))
width = 0.35

plt.bar(x_pos - width/2, acc_values, width, label='Test Accuracy')
plt.bar(x_pos + width/2, balanced_acc_values, width, label='Balanced Accuracy')

plt.xlabel('Optimizer')
plt.ylabel('Accuracy')
plt.title('Optimizer Comparison')
plt.xticks(x_pos, x)
plt.legend()

for i, v in enumerate(acc_values):
    plt.text(i - width/2, v + 0.01, f"{v:.4f}", ha='center')

for i, v in enumerate(balanced_acc_values):
    plt.text(i + width/2, v + 0.01, f"{v:.4f}", ha='center')

plt.ylim(0, 1)
plt.tight_layout()
plt.show()

# ===========TRANSFER LEARNING WITH VGG16 ==============

In [ ]:

print("\n========== STEP 4: TRANSFER LEARNING WITH VGG16 ==========")

# Function to convert grayscale to RGB
def convert_to_rgb(images):
    rgb_images = np.repeat(images, 3, axis=-1)
    return rgb_images


# Convert grayscale images to RGB for VGG16

In [ ]:

X_train_rgb = convert_to_rgb(X_train)
X_val_rgb = convert_to_rgb(X_val)
test_data_rgb = convert_to_rgb(test_data)

# Resize images to 48x48 (to make them compatible with VGG16 while keeping them small)

In [ ]:

def resize_images(images, target_size=(48, 48)):
    resized_images = np.zeros((images.shape[0], target_size[0], target_size[1], images.shape[3]))
    for i, img in enumerate(images):
        resized_images[i] = cv2.resize(img, target_size)
    return resized_images

In [ ]:
X_train_resized = resize_images(X_train_rgb)
X_val_resized = resize_images(X_val_rgb)
test_data_resized = resize_images(test_data_rgb)

# Load VGG16 model without top layers

In [ ]:

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

# Create a new model on top of VGG16

In [ ]:

vgg_model = Sequential()
vgg_model.add(base_model)
vgg_model.add(Flatten())
vgg_model.add(Dense(512, activation='relu'))
vgg_model.add(Dropout(0.5))
vgg_model.add(Dense(num_classes, activation='softmax'))

# Compile the model
vgg_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Display model summary
print("VGG16 Transfer Learning Model Summary:")
vgg_model.summary()

# Train the VGG16 transfer learning model

In [ ]:

vgg_history = vgg_model.fit(
    X_train_resized, y_train,
    batch_size=32,
    epochs=10,
    validation_data=(X_val_resized, y_val),
    callbacks=[early_stopping]
)
# Plot training history
plot_history(vgg_history, title="VGG16 Transfer Learning")


# Evaluate VGG16 transfer learning model

In [ ]:

vgg_test_loss, vgg_test_acc = vgg_model.evaluate(test_data_resized, test_labels)
print(f"VGG16 Transfer Learning - Test accuracy: {vgg_test_acc:.4f}")

In [ ]:
# Make predictions
vgg_y_pred = vgg_model.predict(test_data_resized)
vgg_y_pred_classes = np.argmax(vgg_y_pred, axis=1)

In [ ]:
# Save the VGG16 transfer learning model
vgg_model.save('c:/Users/Acer/Downloads/sign_mnist_test.csv/vgg16_transfer_model.h5')
print("VGG16 transfer learning model saved successfully!")

# ================ FINE-TUNING VGG16 ================

In [ ]:

print("\n========== STEP 5: FINE-TUNING VGG16 ==========")

# Load VGG16 model without top layers for fine-tuning
base_model_ft = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))


In [ ]:
# Unfreeze the last few layers of VGG16
for layer in base_model_ft.layers[:-4]:
    layer.trainable = False
for layer in base_model_ft.layers[-4:]:
    layer.trainable = True

# Create a new model on top of VGG16

In [ ]:

vgg_ft_model = Sequential()
vgg_ft_model.add(base_model_ft)
vgg_ft_model.add(Flatten())
vgg_ft_model.add(Dense(512, activation='relu'))
vgg_ft_model.add(Dropout(0.5))
vgg_ft_model.add(Dense(num_classes, activation='softmax'))

# Compile the model with a lower learning rate
vgg_ft_model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Display model summary
print("VGG16 Fine-Tuning Model Summary:")
vgg_ft_model.summary()

# Train the fine-tuned VGG16 model

In [ ]:

vgg_ft_history = vgg_ft_model.fit(
    X_train_resized, y_train,
    batch_size=32,
    epochs=10,
    validation_data=(X_val_resized, y_val),
    callbacks=[early_stopping]
)
# Plot training history
plot_history(vgg_ft_history, title="VGG16 Fine-Tuning")

# Evaluate fine-tuned VGG16 model

In [ ]:

vgg_ft_acc, vgg_ft_pred_classes = evaluate_model(
    vgg_ft_model, test_data_resized, test_labels, "VGG16 Fine-Tuning"
)


# Visualize some predictions

In [ ]:

plot_sample_predictions(
    test_data,
    [class_names[i] for i in np.argmax(test_labels, axis=1)],
    [class_names[i] for i in vgg_ft_pred_classes],
    model_name="VGG16 Fine-Tuning"
)


In [ ]:
# Save the fine-tuned VGG16 model
vgg_ft_model.save('c:/Users/Acer/Downloads/sign_mnist_test.csv/vgg16_finetuned_model.h5')
print("VGG16 fine-tuned model saved successfully!")


# ======FINAL EVALUATION AND CONCLUSION =======

In [ ]:

print("\n========== STEP 6: FINAL EVALUATION AND CONCLUSION ==========")

# Compare all models
model_names = ['Baseline CNN', 'Deeper CNN', 'CNN with SGD', 'VGG16 Transfer Learning', 'VGG16 Fine-Tuning']
accuracies = [baseline_acc, deeper_acc, sgd_acc, vgg_acc, vgg_ft_acc]

# Plot model comparison

In [ ]:

plt.figure(figsize=(12, 6))
bars = plt.bar(model_names, accuracies, color=['blue', 'green', 'orange', 'red', 'purple'])
plt.title('Model Comparison')
plt.ylabel('Test Accuracy')
plt.ylim(0, 1)
plt.xticks(rotation=45, ha='right')
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.01,
             f"{height:.4f}", ha='center', va='bottom')
plt.tight_layout()
plt.show()

In [ ]:

# Print final comparison
print("\nFinal Model Comparison:")
for name, acc in zip(model_names, accuracies):
    print(f"{name}: {acc:.4f}")


# Determine the best model

In [ ]:

best_model_index = np.argmax(accuracies)
print(f"\nBest Model: {model_names[best_model_index]} with accuracy {accuracies[best_model_index]:.4f}")

# Conclusion


In [ ]:

print("\nConclusion:")
print("1. We implemented and compared multiple CNN architectures for sign language digit recognition.")
print("2. We explored the effects of model depth, regularization techniques, and different optimizers.")
print("3. We applied transfer learning and fine-tuning using the VGG16 architecture.")
print(f"4. The best performing model was {model_names[best_model_index]} with an accuracy of {accuracies[best_model_index]:.4f}.")
print("5. Future work could include exploring other architectures, hyperparameter tuning, and more advanced data augmentation techniques.")